# 1. 리그별 팀 Summary 스탯 크롤링

In [ ]:
import os
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
save_dir = "./Leagues"
targets = ['England-Premier-League', 'Spain-LaLiga', 'Italy-Serie-A', 'Germany-Bundesliga', 'France-Ligue-1']

contents = {'England-Premier-League' : 'standings-20934-content', 
            'Spain-LaLiga' : 'standings-21073-content', 
            'Italy-Serie-A' : 'standings-21087-content', 
            'Germany-Bundesliga' : 'standings-21026-content', 
            'France-Ligue-1' : 'standings-21037-content'}

url = 'https://www.whoscored.com/'
browser = webdriver.Chrome()
browser.get(url)

close_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "webpush-swal2-close")))
close_button.click()

browser.find_element(By.ID, 'Top-Tournaments-btn').click()
tournaments = browser.find_elements(By.CLASS_NAME, 'TournamentNavButton-module_clickableArea__ZFnBl')
# tournaments = [t.get_attribute('href') for t in tournaments] 
# print(tournaments)  

counter = 1
for tournament in tournaments:
    if counter > len(targets):
        break

    link = tournament.get_attribute('href')
    league_name = link.split('/')[-1]

    if league_name in targets:
        if not os.path.exists(f"{save_dir}/{league_name}"): 
            os.makedirs(f"{save_dir}/{league_name}")

        browser2 = webdriver.Chrome()
        wait = WebDriverWait(browser2, 10)
        browser2.get(link)

        close_button = WebDriverWait(browser2, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "webpush-swal2-close")))
        close_button.click()

        select = Select(browser2.find_element(By.ID,'seasons'))
        select.select_by_visible_text("2022/2023")

        if league_name == 'Italy-Serie-A':
            select = Select(browser2.find_element(By.ID,'stages'))
            select.select_by_visible_text("Serie A")

        # time.sleep(1.5)
        table = browser2.find_element(By.ID, contents[league_name])
        rows = table.find_elements(By.TAG_NAME, 'tr')

        dataset = []
        for row in rows:
            team = row.find_element(By.CLASS_NAME, "team-link")
            team_name = team.text
            # print(team_name)
            team_url = team.get_attribute('href')

            num_matches = row.find_element(By.CLASS_NAME, "p").text
            num_win = row.find_element(By.CLASS_NAME, 'w').text
            num_draw = row.find_element(By.CLASS_NAME, 'd').text
            num_lose = row.find_element(By.CLASS_NAME, 'l').text

            ## gf : 득점, ga : 실점, gd : 득실차, pts : 승점
            goal_for = row.find_element(By.CLASS_NAME, 'gf').text
            goal_against = row.find_element(By.CLASS_NAME, 'ga').text
            goal_difference = row.find_element(By.CLASS_NAME, 'gd').text
            points = row.find_element(By.CLASS_NAME, 'pts').text

            dataset.append([team_name, num_matches, num_win, num_draw, num_lose, goal_for, goal_against, goal_difference, points])
        teams_df = pd.DataFrame(dataset, columns=['team_name', 'P', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts'])

        sub_nav = browser2.find_element(By.ID, 'sub-navigation')
        nav_items = sub_nav.find_elements(By.TAG_NAME, 'li')

        ## Team Stat
        team_stat_btn = nav_items[2].click()
        time.sleep(2)

        team_stat_table = browser2.find_element(By.ID, 'top-team-stats-summary-content')
        team_stat_rows = team_stat_table.find_elements(By.TAG_NAME, 'tr')

        total_stats = []
        for idx, row in enumerate(team_stat_rows):
            team_name = row.find_element(By.CLASS_NAME, 'team-link').text.split(' ', 1)[1].strip()
            goal = row.find_element(By.CLASS_NAME, 'goal').text
            # shots_per_game = row.find_element(By.CLASS_NAME, 'shotsPerGame').text
            yellow_cards = row.find_element(By.CLASS_NAME, 'yellow-card-box').text
            red_cards = row.find_element(By.CLASS_NAME, 'red-card-box').text
            possession = row.find_element(By.CLASS_NAME, 'possession').text
            pass_success = row.find_element(By.CLASS_NAME, 'passSuccess').text
            aerial_won_per_game = row.find_element(By.CLASS_NAME, 'aerialWonPerGame').text
            rating = row.find_element(By.CLASS_NAME, 'rating').text

            # total_stats.append([team_name, goal, shots_per_game, yellow_cards, red_cards, possession, pass_success, aerial_won_per_game, rating])
            total_stats.append([team_name, goal, yellow_cards, red_cards, possession, pass_success, aerial_won_per_game, rating])

        # total_stats_df = pd.DataFrame(total_stats, columns=['team_name', 'Goals', 'Shots pg', 'Yellow', 'Red', 'Poss%', 'Pass%', 'A_Won', 'Rating'])
        total_stats_df = pd.DataFrame(total_stats, columns=['team_name', 'Goals', 'Yellow', 'Red', 'Poss%', 'Pass%', 'A_Won', 'Rating'])
        final_df = pd.merge(teams_df, total_stats_df, on='team_name', how='left')
        browser2.close()
        counter += 1

        # final_df.to_csv(f"{save_dir}/{league_name}/{league_name}_22_23_teams_stats.csv")
        final_df.to_csv(f"{save_dir}/{league_name}/22-23_teams_stats.csv")

browser.close()

# 2. 팀별 세부 스탯 크롤링

In [ ]:
save_dir = "./Leagues"

urls = [
    'https://www.whoscored.com/Regions/81/Tournaments/3/Seasons/9120/Stages/21026/TeamStatistics/Germany-Bundesliga-2022-2023',
    'https://www.whoscored.com/Regions/206/Tournaments/4/Seasons/9149/Stages/21073/TeamStatistics/Spain-LaLiga-2022-2023',
    'https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/9129/Stages/21037/TeamStatistics/France-Ligue-1-2022-2023',
    'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/9075/Stages/20934/TeamStatistics/England-Premier-League-2022-2023',
    'https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/9159/Stages/21087/TeamStatistics/Italy-Serie-A-2022-2023',
    ]


teams_df_list = [
    f"{save_dir}/Germany-Bundesliga/22-23_teams_stats.csv",
    f"{save_dir}/Spain-LaLiga/22-23_teams_stats.csv",
    f"{save_dir}/France-Ligue-1/22-23_teams_stats.csv",
    f"{save_dir}/England-Premier-League/22-23_teams_stats.csv",
    f"{save_dir}/Italy-Serie-A/22-23_teams_stats.csv"
]

In [ ]:
for idx, url in enumerate(urls):
    league_name = teams_df_list[idx].split('/')[-2]
    print(url, f"{save_dir}/{league_name}/{league_name}")

    team_df = pd.read_csv(teams_df_list[idx])
    final_df = team_df

    browser = webdriver.Chrome()
    browser.get(url)
    wait = WebDriverWait(browser, 10)

    close_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "webpush-swal2-close")))
    close_button.click()

    options_container = wait.until(EC.presence_of_element_located((By.ID, 'stage-team-stats-options')))
    list_items = options_container.find_elements(By.TAG_NAME, 'li')

    defensive_stat = []
    offensive_stat = []
    xg_stat = []
    detailed_stat = []
    for list_item in list_items[1:-1]:
        item_name = list_item.text
        # browser.execute_script("arguments[0].click();", list_item)
        list_item.click()
        time.sleep(3)

        container = browser.find_element(By.ID, f'stage-team-stats-{item_name.lower()}')
        print(f'stage-team-stats-{item_name.lower()}')

        table = container.find_element(By.CLASS_NAME, 'with-centered-columns')
        # headers = table.find_elements(By.TAG_NAME, 'th')
        # columns = [header.text for header in headers]
        time.sleep(3)

        body = table.find_element(By.ID, 'top-team-stats-summary-content')
        rows = body.find_elements(By.TAG_NAME, 'tr')

        if item_name == "Defensive":
            data = []
            # columns = ['team_name', 'Shoted pg', 'Tackles pg', 'Intercept pg', 'Fouls pg', 'Offsides pg']
            columns = ['team_name', 'Shot allowed', 'Tackles pg', 'Intercept pg', 'Fouls pg', 'Offsides pg']
            for row in rows:
                team_name = row.find_element(By.CLASS_NAME, 'team-link').text.split(' ', 1)[1].strip()
                # shots_pg = row.find_element(By.CLASS_NAME, 'shotsConcededPerGame').text
                shots_allowed = row.find_element(By.CLASS_NAME, 'shotsConcededPerGame').text
                tackles_pg = row.find_element(By.CLASS_NAME, 'tacklePerGame').text
                interceptions_pg = row.find_element(By.CLASS_NAME, 'interceptionPerGame').text
                fouls_pg = row.find_element(By.CLASS_NAME, 'foulsPerGame').text
                offsides_pg = row.find_element(By.CLASS_NAME, 'offsideGivenPerGame').text

                # data.append([team_name, shots_pg, tackles_pg, interceptions_pg, fouls_pg, offsides_pg])
                data.append([team_name, shots_allowed, tackles_pg, interceptions_pg, fouls_pg, offsides_pg])
            defensive_stat = pd.DataFrame(data, columns=columns)
            final_df = pd.merge(final_df, defensive_stat, on='team_name', how='left')

        if item_name == 'Offensive':
            data = []
            columns = ['team_name', 'Shots OT pg', 'Dribbles pg', 'Fouled pg']
            for row in rows:
                team_name = row.find_element(By.CLASS_NAME, 'team-link').text.split(' ', 1)[1].strip()
                shot_on_target_per_game = row.find_element(By.CLASS_NAME, 'shotOnTargetPerGame').text
                dribble_won_per_game = row.find_element(By.CLASS_NAME, 'shotOnTargetPerGame').text
                foul_given_per_game = row.find_element(By.CLASS_NAME, 'shotOnTargetPerGame').text

                data.append([team_name, shot_on_target_per_game, dribble_won_per_game, foul_given_per_game])
            offensive_stat = pd.DataFrame(data, columns=columns)
            final_df = pd.merge(final_df, offensive_stat, on='team_name', how='left')
        
        if item_name == 'xG':
            data = []
            # columns = ['team_name', 'xG', 'Goals', 'xGDiff', 'Shots', 'xG/shots']
            columns = ['team_name', 'xG', 'GoalsDiff', 'xGDiff', 'Shots', 'xG/shots']
            for row in rows:
                team_name = row.find_element(By.CLASS_NAME, 'team-link').text.split(' ', 1)[1].strip()
                xg = row.find_element(By.CLASS_NAME, 'xG').text
                # goal_exc_own = row.find_element(By.CLASS_NAME, 'goalExcOwn').text
                goals_diff = row.find_element(By.CLASS_NAME, 'goalExcOwn').text
                xgdiff = row.find_element(By.CLASS_NAME, 'xGDiff').text
                total_shots = row.find_element(By.CLASS_NAME, 'totalShots').text
                xg_per_shot = row.find_element(By.CLASS_NAME, 'xGPerShot').text

                # data.append([team_name, xg, goal_exc_own, xgdiff, total_shots, xg_per_shot])
                data.append([team_name, xg, goals_diff, xgdiff, total_shots, xg_per_shot])
            xg_stat = pd.DataFrame(data, columns=columns)
            final_df = pd.merge(final_df, xg_stat, on='team_name', how='left')

        # if item_name == "Detailed":
        #     data = []
        #     columns = ['team_name', 'shotsTotal', 'shot0boxTotal', 'shotSixYardBox', 'shotPentaltyArea']
        #     for row in rows:
        #         team_name = row.find_element(By.CLASS_NAME, 'team-link').text.split(' ', 1)[1].strip()
        #         shots_total = row.find_element(By.CLASS_NAME, 'shotsTotal').text
        #         shots_0box_total = row.find_element(By.CLASS_NAME, 'shotOboxTotal').text
        #         shots_six_yard_box = row.find_element(By.CLASS_NAME, 'shotSixYardBox').text
        #         shots_penalty_area = row.find_element(By.CLASS_NAME, 'shotPenaltyArea').text

        #         data.append([team_name, shots_total, shots_0box_total, shots_six_yard_box, shots_penalty_area])
        #     detailed_stat = pd.DataFrame(data, columns=columns)
        #     final_df = pd.merge(final_df, detailed_stat, on='team_name', how='left')

    final_df.to_csv(f"{save_dir}/{league_name}/22-23_teams_stats.csv")
    browser.close()

# 3.팀별 선수 스탯 크롤링

In [ ]:
import os
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

save_dir = "./Leagues"
targets = ['England-Premier-League', 'Spain-LaLiga', 'Italy-Serie-A', 'Germany-Bundesliga', 'France-Ligue-1']
urls = [
    'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/9075/Stages/20934/PlayerStatistics/England-Premier-League-2022-2023',
    'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/9149/Stages/21073/PlayerStatistics/Spain-LaLiga-2022-2023',
    'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/9159/Stages/21087/PlayerStatistics/Italy-Serie-A-2022-2023',
    'https://1xbet.whoscored.com/Regions/81/Tournaments/3/Seasons/9120/Stages/21026/PlayerStatistics/Germany-Bundesliga-2022-2023',
    'https://1xbet.whoscored.com/Regions/74/Tournaments/22/Seasons/9129/Stages/21037/PlayerStatistics/France-Ligue-1-2022-2023'
]


for idx, url in enumerate(urls):
    league_name = targets[idx]

    browser = webdriver.Chrome()
    browser.get(url)

    close_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "webpush-swal2-close")))
    close_button.click()

    sub_nav = browser.find_element(By.ID, 'sub-navigation')
    nav_items = sub_nav.find_elements(By.TAG_NAME, 'li')
    nav_items[3].click()
    time.sleep(1.5)

    grid_toolbar = browser.find_element(By.CLASS_NAME, 'grid-toolbar')
    grid_apps = grid_toolbar.find_element(By.ID, 'apps')
    grid_btns = grid_apps.find_elements(By.TAG_NAME, 'dd')
    grid_btns[1].click()
    time.sleep(1.5)

    # list_boxes = browser.find_element(By.ID, 'statistics-paging-summary')
    # right_box = list_boxes.find_element(By.CLASS_NAME, 'right')
    # right_box_line = right_box.find_element(By.TAG_NAME, 'dt').text
    # print(right_box_line)

    current_page = int(browser.find_element(By.ID, "currentPage").get_attribute('value'))
    total_pages = int(browser.find_element(By.ID, "totalPages").get_attribute('value')) + 1
    print(current_page, total_pages)

    total_players = []
    while current_page < total_pages:
        time.sleep(2)
        players_table = browser.find_element(By.ID, 'player-table-statistics-body')
        players = players_table.find_elements(By.TAG_NAME, 'tr')

        for player in players:
            info_box = player.find_element(By.CLASS_NAME, 'overflow-text')
            infos = info_box.find_elements(By.TAG_NAME, 'span')
            infos = [info.text.strip() for info in infos]
            
            player_name = infos[0]
            team_name = info_box.find_element(By.CLASS_NAME, 'team-name').text.split(',')[0].strip()
            age_pos = infos[3].split(',', 1)
            age = age_pos[0]
            position = age_pos[1]

            table_data = player.find_elements(By.TAG_NAME, 'td')
            table_data = [data.text.strip() if not data.text == '-' else 0 for data in table_data]
            
            apps = table_data[2]
            mins = table_data[3]
            goals = table_data[4]
            assists = table_data[5]
            yellow_cards = table_data[6]
            red_cards = table_data[7]
            shots_per_game = table_data[8]
            pass_success = table_data[9]
            aerials_won = table_data[10]
            mom = table_data[11]
            rating = table_data[12]

            print(player_name, team_name, age, position, apps, mins, goals, assists, yellow_cards, red_cards, shots_per_game, pass_success, aerials_won, mom, rating)
            total_players.append([player_name, team_name, age, position, apps, mins, goals, assists, yellow_cards, red_cards, shots_per_game, pass_success, aerials_won, mom, rating])

        next_button = browser.find_element(By.ID, "next")
        if next_button.is_enabled():
            next_button.click()
            current_page += 1

    summary_df = pd.DataFrame(total_players, columns=['player_name', 'team_name', 'age', 'position', 'Apps', 'Mins', 'Goals', 'Assists', 'Yel', 'Red', 'SpG', 'PS%', 'AerialsWon', 'MoM', 'Rating'])
    summary_df.to_csv(f'./{save_dir}/{league_name}/22-23_players_stats.csv')
    browser.close()

# 4. 선수별 세부 스탯 크롤링

In [ ]:
import os
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

save_dir = "./Leagues"
targets = ['England-Premier-League', 'Spain-LaLiga', 'Italy-Serie-A', 'Germany-Bundesliga', 'France-Ligue-1']
urls = [
    'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/9075/Stages/20934/PlayerStatistics/England-Premier-League-2022-2023',
    'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/9149/Stages/21073/PlayerStatistics/Spain-LaLiga-2022-2023',
    'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/9159/Stages/21087/PlayerStatistics/Italy-Serie-A-2022-2023',
    'https://1xbet.whoscored.com/Regions/81/Tournaments/3/Seasons/9120/Stages/21026/PlayerStatistics/Germany-Bundesliga-2022-2023',
    'https://1xbet.whoscored.com/Regions/74/Tournaments/22/Seasons/9129/Stages/21037/PlayerStatistics/France-Ligue-1-2022-2023'
]

for idx, url in enumerate(urls):
    league_name = targets[idx]
    browser = webdriver.Chrome()
    browser.get(url)

    close_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "webpush-swal2-close")))
    close_button.click()


    # player_stats_df = pd.read_csv(f'{save_dir}/{league_name}/{league_name}-22_23_player_stats.csv')
    player_stats_df = pd.read_csv(f'{save_dir}/{league_name}/22-23_players_stats.csv')
    final_df = player_stats_df

    ## Player Stat
    sub_nav = browser.find_element(By.ID, 'sub-navigation')
    nav_items = sub_nav.find_elements(By.TAG_NAME, 'li')
    nav_items[3].click()
    time.sleep(2)

    data = []
    options = browser.find_element(By.ID, 'stage-top-player-stats-options').find_elements(By.TAG_NAME, 'li')
    for option in options[1:-1]:
        option.click()
        time.sleep(2)

        grid_container = browser.find_element(By.ID, f'statistics-mini-filter-{option.text.lower()}')
        grid_toolbar = grid_container.find_element(By.CLASS_NAME, 'grid-toolbar')
        grid_apps = grid_toolbar.find_element(By.ID, 'apps')
        grid_btns = grid_apps.find_elements(By.TAG_NAME, 'dd')
        target_btn = grid_btns[1].find_element(By.TAG_NAME, 'a')
        target_btn.click()
        time.sleep(2)

        paging_container = browser.find_element(By.ID, f'statistics-paging-{option.text.lower()}')
        current_page = int(paging_container.find_element(By.ID, "currentPage").get_attribute('value'))
        total_pages = int(paging_container.find_element(By.ID, "totalPages").get_attribute('value')) + 1
        print(current_page, total_pages)

        total_players = []
        while current_page < total_pages:
            time.sleep(2)
            container = browser.find_element(By.ID, f'statistics-table-{option.text.lower()}')
            players_table = container.find_element(By.ID, 'player-table-statistics-body')
            players = players_table.find_elements(By.TAG_NAME, 'tr')

            for player in players:
                info_box = player.find_element(By.CLASS_NAME, 'overflow-text')
                infos = info_box.find_elements(By.TAG_NAME, 'span')
                infos = [info.text.strip() for info in infos]
                
                player_name = infos[0]
                team_name = infos[1]
                age_pos = infos[3].split(',', 1)
                age = age_pos[0]
                position = age_pos[1]

                table_data = player.find_elements(By.TAG_NAME, 'td')
                table_data = [data.text.strip() if not data.text == '-' else 0 for data in table_data]
                
                if option.text == 'Defensive':
                    tackle_per_game = player.find_element(By.CLASS_NAME, 'tacklePerGame').text if player.find_element(By.CLASS_NAME, 'tacklePerGame').text != '-' else 0
                    interception_per_game = player.find_element(By.CLASS_NAME, 'interceptionPerGame').text if player.find_element(By.CLASS_NAME, 'interceptionPerGame').text != '-' else 0
                    fouls_per_game = player.find_element(By.CLASS_NAME, 'foulsPerGame').text if player.find_element(By.CLASS_NAME, 'foulsPerGame').text != '-' else 0
                    offside_won_per_game = player.find_element(By.CLASS_NAME, 'offsideWonPerGame').text if player.find_element(By.CLASS_NAME, 'offsideWonPerGame').text != '-' else 0
                    clearance_per_game = player.find_element(By.CLASS_NAME, 'clearancePerGame').text if player.find_element(By.CLASS_NAME, 'clearancePerGame').text != '-' else 0
                    # was_dribble_per_game = player.find_element(By.CLASS_NAME, 'wasDribbledPerGame').text if player.find_element(By.CLASS_NAME, 'wasDribbledPerGame').text != '-' else 0
                    dribbles_allowed = player.find_element(By.CLASS_NAME, 'wasDribbledPerGame').text if player.find_element(By.CLASS_NAME, 'wasDribbledPerGame').text != '-' else 0
                    outfielder_block_game = player.find_element(By.CLASS_NAME, 'outfielderBlockPerGame').text if player.find_element(By.CLASS_NAME, 'outfielderBlockPerGame').text != '-' else 0
                    goalown = player.find_element(By.CLASS_NAME, 'goalOwn').text if player.find_element(By.CLASS_NAME, 'goalOwn').text != '-' else 0

                    # total_players.append([player_name, tackle_per_game, interception_per_game, fouls_per_game, offside_won_per_game, clearance_per_game, was_dribble_per_game, outfielder_block_game, goalown])
                    # print(player_name, tackle_per_game, interception_per_game, fouls_per_game, offside_won_per_game, clearance_per_game, was_dribble_per_game, outfielder_block_game, goalown)
                    total_players.append([player_name, tackle_per_game, interception_per_game, fouls_per_game, offside_won_per_game, clearance_per_game, dribbles_allowed, outfielder_block_game, goalown])
                    print(player_name, tackle_per_game, interception_per_game, fouls_per_game, offside_won_per_game, clearance_per_game, dribbles_allowed, outfielder_block_game, goalown)

                if option.text == 'Offensive':
                    key_pass_per_game = player.find_element(By.CLASS_NAME, 'keyPassPerGame').text if player.find_element(By.CLASS_NAME, 'keyPassPerGame').text != '-' else 0
                    # dribble_won_per_game = player.find_element(By.CLASS_NAME, 'dribbleWonPerGame').text if player.find_element(By.CLASS_NAME, 'dribbleWonPerGame').text != '-' else 0
                    dribble = player.find_element(By.CLASS_NAME, 'dribbleWonPerGame').text if player.find_element(By.CLASS_NAME, 'dribbleWonPerGame').text != '-' else 0
                    foul_given_game = player.find_element(By.CLASS_NAME, 'foulGivenPerGame').text if player.find_element(By.CLASS_NAME, 'foulGivenPerGame').text != '-' else 0
                    offside_given_per_game = player.find_element(By.CLASS_NAME, 'offsideGivenPerGame').text if player.find_element(By.CLASS_NAME, 'offsideGivenPerGame').text != '-' else 0
                    dispossessed_per_game = player.find_element(By.CLASS_NAME, 'dispossessedPerGame').text if player.find_element(By.CLASS_NAME, 'dispossessedPerGame').text != '-' else 0
                    turnover_per_game = player.find_element(By.CLASS_NAME, 'turnoverPerGame').text if player.find_element(By.CLASS_NAME, 'turnoverPerGame').text != '-' else 0

                    # total_players.append([player_name, key_pass_per_game, dribble_won_per_game, foul_given_game, offside_given_per_game, dispossessed_per_game, turnover_per_game])
                    # print(player_name, key_pass_per_game, dribble_won_per_game, foul_given_game, offside_given_per_game, dispossessed_per_game, turnover_per_game)
                    total_players.append([player_name, key_pass_per_game, dribble, foul_given_game, offside_given_per_game, dispossessed_per_game, turnover_per_game])
                    print(player_name, key_pass_per_game, dribble, foul_given_game, offside_given_per_game, dispossessed_per_game, turnover_per_game)

                if option.text == 'Passing':
                    average_per_pass = player.find_element(By.CLASS_NAME, 'totalPassesPerGame').text if player.find_element(By.CLASS_NAME, 'totalPassesPerGame').text != '-' else 0
                    crosses_per_game = player.find_element(By.CLASS_NAME, 'accurateCrossesPerGame').text if player.find_element(By.CLASS_NAME, 'accurateCrossesPerGame').text != '-' else 0
                    longpass_per_game = player.find_element(By.CLASS_NAME, 'accurateLongPassPerGame').text if player.find_element(By.CLASS_NAME, 'accurateLongPassPerGame').text != '-' else 0
                    through_ball_per_game = player.find_element(By.CLASS_NAME, 'accurateThroughBallPerGame').text if player.find_element(By.CLASS_NAME, 'accurateThroughBallPerGame').text != '-' else 0

                    total_players.append([player_name, average_per_pass, crosses_per_game, longpass_per_game, through_ball_per_game])
                    print(player_name, average_per_pass, crosses_per_game, longpass_per_game, through_ball_per_game)

                if option.text == 'xG':
                    xG = player.find_element(By.CLASS_NAME, 'xG').text if player.find_element(By.CLASS_NAME, 'xG').text != '-' else 0
                    xGDiff = player.find_element(By.CLASS_NAME, 'xGDiff').text if player.find_element(By.CLASS_NAME, 'xGDiff').text != '-' else 0
                    xG_per_ninety = player.find_element(By.CLASS_NAME, 'xGPerNinety').text if player.find_element(By.CLASS_NAME, 'xGPerNinety').text != '-' else 0
                    total_shots = player.find_element(By.CLASS_NAME, 'totalShots').text if player.find_element(By.CLASS_NAME, 'totalShots').text != '-' else 0
                    xG_per_shot = player.find_element(By.CLASS_NAME, 'xGPerShot').text if player.find_element(By.CLASS_NAME, 'xGPerShot').text != '-' else 0

                    total_players.append([player_name, xG, xGDiff, xG_per_ninety, total_shots, xG_per_shot])
                    print(player_name, xG, xGDiff, xG_per_ninety, total_shots, xG_per_shot)


            grid_toolbar = browser.find_element(By.ID, f'statistics-paging-{option.text.lower()}').find_element(By.CLASS_NAME, 'grid-toolbar')
            rigth_box = grid_toolbar.find_element(By.CLASS_NAME, 'right')
            next_button = rigth_box.find_element(By.ID, "next")
            if next_button.is_enabled():
                next_button.click()
                current_page += 1
        
        if option.text == 'Defensive':
            # df = pd.DataFrame(total_players, columns=['player_name', 'Tackles', 'Inter', 'Fouls', 'Offsides', 'Clear', 'Was_dribble', 'Blocks', 'OwnGoal'])
            df = pd.DataFrame(total_players, columns=['player_name', 'Tackles', 'Inter', 'Fouls', 'Offsides', 'Clear', 'Dribbles allowed', 'Blocks', 'OwnGoal'])
            final_df = pd.merge(final_df, df, on='player_name', how='left')

        if option.text == 'Offensive':
            # df = pd.DataFrame(total_players, columns=['player_name', 'key_pass_per_game', 'dribble_won_per_game', 'foul_given_game', 'offside_given_per_game', 'dispossessed_per_game', 'turnover_per_game'])
            df = pd.DataFrame(total_players, columns=['player_name', 'Key_pass_per_game', 'Dribble', 'Foul_given_game', 'Offside_given_per_game', 'Dispossessed_per_game', 'Unsuccessful touches'])
            final_df = pd.merge(final_df, df, on='player_name', how='left')

        if option.text == 'Passing':
            df = pd.DataFrame(total_players, columns=['player_name', 'Average_per_pass', 'Crosses_per_game', 'Longpass_per_game', 'Through_ball_per_game'])
            final_df = pd.merge(final_df, df, on='player_name', how='left')

        if option.text == 'xG':
            df = pd.DataFrame(total_players, columns=['player_name', 'xG', 'xGDiff', 'xGPerNinety', 'totalShots', 'xGPerShot'])
            final_df = pd.merge(final_df, df, on='player_name', how='left')

    final_df.to_csv(f'{save_dir}/{league_name}/22-23_players_stats.csv')
    browser.close()


# 5.션수별 Detailed 항목 데이터 크롤링

In [1]:
import os
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

detail_select_box = [
    'Tackles', 
    'Interception', 
    'Fouls', 
    'Cards', 
    'Offsides', 
    'Clearances', 
    'Blocks', 
    'Saves', 
    'Shots', 
    'Goals', 
    'Dribbles', 
    'Possession loss', 
    'Aerial',
    'Passes', 
    'Key passes', 
    'Assists'
]

save_dir = "./Leagues"
targets = [
    'England-Premier-League', 
    'Spain-LaLiga', 
    'Italy-Serie-A', 
    'Germany-Bundesliga', 
    'France-Ligue-1'
]

urls = [
    'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/9075/Stages/20934/PlayerStatistics/England-Premier-League-2022-2023',
    'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/9149/Stages/21073/PlayerStatistics/Spain-LaLiga-2022-2023',
    'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/9159/Stages/21087/PlayerStatistics/Italy-Serie-A-2022-2023',
    'https://1xbet.whoscored.com/Regions/81/Tournaments/3/Seasons/9120/Stages/21026/PlayerStatistics/Germany-Bundesliga-2022-2023',
    'https://1xbet.whoscored.com/Regions/74/Tournaments/22/Seasons/9129/Stages/21037/PlayerStatistics/France-Ligue-1-2022-2023'
]

In [2]:
for idx, url in enumerate(urls):
    league_name = targets[idx]
    browser = webdriver.Chrome()
    wait_timer = WebDriverWait(browser, 5)
    browser.get(url)

    if league_name == 'Italy-Serie-A':
        select_element = wait_timer.until(EC.element_to_be_clickable((By.ID, 'stages')))
        select_box = Select(select_element)
        select_box.select_by_visible_text("Serie A")  

    ## Player Stat
    sub_nav = browser.find_element(By.ID, 'sub-navigation')
    nav_items = sub_nav.find_elements(By.TAG_NAME, 'li')
    nav_items[3].click()

    ## Options
    options = browser.find_element(By.ID, 'stage-top-player-stats-options').find_elements(By.TAG_NAME, 'li')
    option = options[-1]
    option.click()
    time.sleep(5)

    filter_container = browser.find_element(By.ID, 'statistics-filter-detailed')
    select_box = Select(filter_container.find_element(By.ID, 'category'))
    for selected in detail_select_box:
        print(selected)
        select_box.select_by_visible_text(selected)
        time.sleep(5)

        paging_container = browser.find_element(By.ID, f'statistics-paging-{option.text.lower()}')
        current_page = int(paging_container.find_element(By.ID, "currentPage").get_attribute('value'))
        total_pages = int(paging_container.find_element(By.ID, "totalPages").get_attribute('value'))
        # total_pages = 2

        if current_page != 1:
            grid_toolbar = browser.find_element(By.ID, f'statistics-paging-{option.text.lower()}').find_element(By.CLASS_NAME, 'grid-toolbar')
            rigth_box = grid_toolbar.find_element(By.CLASS_NAME, 'right')
            first_button = rigth_box.find_element(By.ID, "first")
            first_button.click()

        container = browser.find_element(By.ID, 'statistics-table-detailed')
        table = container.find_element(By.ID, 'top-player-stats-summary-grid')

        ## Columns
        thead = table.find_element(By.TAG_NAME, 'thead')
        columns = thead.find_elements(By.TAG_NAME, 'th')[4:-1] ## apps, mins, rating 컬럼 제외.
        columns = [column.text.strip() for column in columns]
        for idx, column in enumerate(columns):
            if column == 'Total':
                columns[idx] = f'Total_{selected}'
            if column == 'SixYardBox':
                columns[idx] = f'SixYardBox_{selected}'
            if column == 'PenaltyArea':
                columns[idx] = f'PenaltyArea_{selected}'
            if column == 'OutOfBox':
                columns[idx] = f'OutOfBox_{selected}'
        columns = ['player_name', 'team_name'] + columns
        print(', '.join(columns))

        total_data = []
        while current_page <= total_pages:
            time.sleep(5)
            container = browser.find_element(By.ID, f'statistics-table-{option.text.lower()}')
            players_table = container.find_element(By.ID, 'player-table-statistics-body')
            players = players_table.find_elements(By.TAG_NAME, 'tr')

            for player in players:
                info_box = player.find_element(By.CLASS_NAME, 'overflow-text')
                infos = info_box.find_elements(By.TAG_NAME, 'span')
                infos = [info.text.strip() for info in infos]
                
                player_name = infos[0]
                team_name = infos[2][:-1]
                age_pos = infos[3].split(',', 1)
                age = age_pos[0]
                position = age_pos[1]

                tds = player.find_elements(By.TAG_NAME, 'td')[4:-1]
                tds = [td.text.strip() for td in tds]
                tds = ['0' if x == '-' else x for x in tds]
                tds = [player_name, team_name] + tds
                print(f"  {', '.join(tds)}")
                total_data.append(tds)

            grid_toolbar = browser.find_element(By.ID, f'statistics-paging-{option.text.lower()}').find_element(By.CLASS_NAME, 'grid-toolbar')
            rigth_box = grid_toolbar.find_element(By.CLASS_NAME, 'right')
            next_button = rigth_box.find_element(By.ID, "next")
            if next_button.is_enabled():
                next_button.click()
                current_page += 1  

        print()
        df = pd.DataFrame(total_data, columns=columns)
        df.to_csv(f'{save_dir}/{league_name}/22-23_players_{selected}.csv', index=False)
        # break
    # break

browser.close()

Tackles
player_name, team_name, TotalTackles, DribbledPast, TotalAttemptedTackles
  Asmir Begovic, Everton, 0, 0, 0
  Kevin De Bruyne, Manchester City, 0.9, 0.8, 1.7
  Stefan Ortega, Manchester City, 0, 0, 0
  Erling Haaland, Manchester City, 0.1, 0.1, 0.2
  Harry Kane, Tottenham, 0.4, 0.4, 0.8
  Bruno Fernandes, Manchester United, 1.8, 1.1, 3
  Gabriel Jesus, Arsenal, 1.3, 0.7, 1.9
  Kieran Trippier, Newcastle, 1.9, 1.2, 3
  Bukayo Saka, Arsenal, 1.7, 0.6, 2.3
  Solly March, Brighton, 1.6, 0.6, 2.2
  Jack Grealish, Manchester City, 0.9, 0.4, 1.3
  Martin Ødegaard, Arsenal, 1, 0.9, 1.9
  Rodri, Manchester City, 1.9, 0.8, 2.6
  Gabriel Martinelli, Arsenal, 0.9, 0.4, 1.2
  Ivan Toney, Brentford, 0.7, 0.5, 1.2
  Joelinton, Newcastle, 2.2, 1.1, 3.3
  James Maddison, Leicester, 1.7, 1, 2.7
  Rodrigo Bentancur, Tottenham, 2.5, 0.7, 3.2
  Bruno Guimarães, Newcastle, 2.4, 0.8, 3.3
  Casemiro, Manchester United, 3.1, 1.6, 4.8
  Mohamed Salah, Liverpool, 0.6, 0.4, 1
  Phil Foden, Manchester City